In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("test")
         .getOrCreate())

In [3]:
ratings_rdd = sc.textFile('/labs/laba01/ml-100k/u.data').map(lambda x: x.split('\t')[1:3])

In [4]:
def plus(x, y):
    return x + y

def calc_ratings(rdd):
    return rdd.map(lambda x: (x[1],1)).reduceByKey(plus).sortBy(lambda x: x[0]).map(lambda x: x[1]).collect()

In [5]:
ratings = {
    'hist_film' : calc_ratings(ratings_rdd.filter(lambda x: x[0] == '318')),
    'hist_all' : calc_ratings(ratings_rdd)
}

In [6]:
ratings

{'hist_film': [4, 6, 23, 79, 186],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [73]:
with open('/data/home/roman.razumovskiy/lab01.json', 'w') as ratings_json:
    json.dump(ratings, ratings_json)

In [28]:
spark.stop()